In [1]:
from lephare import *
import numpy as np
from matplotlib import pylab as plt
from scipy import integrate as sciint
import glob, time
%matplotlib inline

In [2]:
allFlt = FilterSvc.from_config("../examples/COSMOS.para")

In [3]:
sed = StarSED("test", 0)
sed.read("../sed/STAR/BD_NEW/lte012.0-4.0-0.0a+0.0.BT-Settl.spec.txt")

In [4]:
opavec = GalMag.read_opa()

In [5]:
mag = MagSvc.from_config("Star","../examples/COSMOS.para")

Number of keywords read in the config file: 92


In [6]:
newsed = mag.make_maglib(sed)

In [ ]:
newsed.mag